I've made edits to the CONTROL code which makes it so that it appends all data from every Match, Series, Player, and Team.
Let's see what kind of interesting queries we can run on this.

In [17]:
#Import necessary modules
import sqlite3
import pandas as pd
import sys
#Now let's make a custom query function to make things easier.
def QUERY(sql, connect=sqlite3.connect('C:/Users/carte/OneDrive/CONTROL(onedrive)/CONTROL/ControlDataBase.db'), params=None, is_select=False):
    if is_select:
        if params:
            return pd.read_sql_query(sql, connect, params=params)
        else:
            return pd.read_sql_query(sql, connect)
    else:
        cur = connect.cursor()
        if params:
            cur.execute(sql, params)
        else:
            cur.execute(sql)
        connect.commit()
        return cur.lastrowid

In [30]:
#First, let's see the basic structure of all of the tables here by creating a function to show the schema of every table.
def print_table_schemas(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table_name in tables:
        if table_name[0] != 'Game' and table_name[0] != 'sqlite_sequence': #Game Table is currently disabled, as it nearly triples CONTROL runtime.
                                 #Following testing to confirm it works, I elected to not store Game data to speed things up.
            table_name = table_name[0]
            print(f"Schema of '{table_name}':")
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            for column in columns:
                print(column)
            print("\n")
    cursor.close()
    conn.close()
    
print_table_schemas("C:/Users/carte/OneDrive/CONTROL(onedrive)/CONTROL/ControlDataBase.db")

Schema of 'Team':
(0, 'team_id', 'INTEGER', 0, None, 1)
(1, 'team_name', 'TEXT', 0, None, 0)
(2, 'region_of_origin', 'TEXT', 0, None, 0)
(3, 'season_of_origin', 'INTEGER', 0, None, 0)


Schema of 'Player':
(0, 'player_id', 'INTEGER', 0, None, 1)
(1, 'player_name', 'TEXT', 0, None, 0)
(2, 'amp', 'REAL', 0, None, 0)
(3, 'season_of_origin', 'INTEGER', 0, None, 0)


Schema of 'Match':
(0, 'match_id', 'INTEGER', 0, None, 1)
(1, 'winning_team_id', 'INTEGER', 0, None, 0)
(2, 'losing_team_id', 'INTEGER', 0, None, 0)
(3, 'match_score', 'TEXT', 0, None, 0)
(4, '_0N_winner_id', 'INTEGER', 0, None, 0)
(5, '_1N_winner_id', 'INTEGER', 0, None, 0)
(6, '_2N_winner_id', 'INTEGER', 0, None, 0)
(7, '_3N_winner_id', 'INTEGER', 0, None, 0)
(8, '_4N_winner_id', 'INTEGER', 0, None, 0)
(9, '_5N_winner_id', 'INTEGER', 0, None, 0)
(10, '_6N_winner_id', 'INTEGER', 0, None, 0)
(11, '_7N_winner_id', 'INTEGER', 0, None, 0)
(12, '_8N_winner_id', 'INTEGER', 0, None, 0)
(13, 'playoffs', 'TEXT', 0, None, 0)


Schema of

In [19]:
#As another test, let's see a few of the Match values.
query = """
SELECT * FROM Match LIMIT 10;
"""
QUERY(query, is_select=True)

,match_id,winning_team_id,losing_team_id,match_score,_0N_winner_id,_1N_winner_id,_2N_winner_id,_3N_winner_id,_4N_winner_id,_5N_winner_id,_6N_winner_id,_7N_winner_id,_8N_winner_id,playoffs
0,1,2,1,5-3,2,2,2,1,1,1,2,2,None,No
1,2,3,1,7-1,3,3,1,3,3,3,3,3,None,No
2,3,1,4,5-3,1,1,4,4,1,1,1,4,None,No
3,4,1,5,6-2,5,5,1,1,1,1,1,1,None,No
4,5,1,6,5-3,6,1,1,1,6,1,6,1,None,No
5,6,1,7,5-3,1,7,1,7,1,1,1,7,None,No
6,7,8,1,5-3,1,8,1,8,8,8,8,1,None,No
7,8,9,1,5-3,9,9,9,1,9,1,1,9,None,No
8,9,1,10,5-3,10,10,1,1,1,1,1,10,None,No
9,10,1,11,5-3,11,11,1,1,1,1,11,1,None,No


In [20]:
#How about only the matches where the score was 8-0?
#Let's see how many there are first.
query = """
SELECT COUNT(match_id) FROM Match WHERE match_score = '8-0';
"""
QUERY(query, is_select=True)

,COUNT(match_id)
0,2645


In [22]:
#Since there are so many, let's only look at the first 10.
query = """
SELECT * FROM Match WHERE match_score = '8-0' LIMIT 10;
"""
QUERY(query, is_select=True)

,match_id,winning_team_id,losing_team_id,match_score,_0N_winner_id,_1N_winner_id,_2N_winner_id,_3N_winner_id,_4N_winner_id,_5N_winner_id,_6N_winner_id,_7N_winner_id,_8N_winner_id,playoffs
0,21,22,1,8-0,22,22,22,22,22,22,22,22,None,No
1,28,2,5,8-0,2,2,2,2,2,2,2,2,None,No
2,31,2,8,8-0,2,2,2,2,2,2,2,2,None,No
3,60,3,14,8-0,3,3,3,3,3,3,3,3,None,No
4,94,26,4,8-0,26,26,26,26,26,26,26,26,None,No
5,100,11,5,8-0,11,11,11,11,11,11,11,11,None,No
6,115,26,5,8-0,26,26,26,26,26,26,26,26,None,No
7,123,6,14,8-0,6,6,6,6,6,6,6,6,None,No
8,129,6,20,8-0,6,6,6,6,6,6,6,6,None,No
9,189,26,9,8-0,26,26,26,26,26,26,26,26,None,No


In [23]:
#I also want to see the team names, but the only identifier is their team_id, which doesn't help me.
#Let's write a JOIN query to get the names of the winning teams as well.
query = """
SELECT winning_team_id, team_name, match_score FROM Match LEFT JOIN Team ON Match.winning_team_id = Team.team_id;
"""
QUERY(query, is_select=True)

,winning_team_id,team_name,match_score
0,2,Universal_Aether,5-3
1,3,Universal_Oracle,7-1
2,1,Universal_Arch-Kings,5-3
3,1,Universal_Arch-Kings,6-2
4,1,Universal_Arch-Kings,5-3
...,...,...,...
166305,328,Ice-Wall_Leviathan,6-2
166306,315,Web-of-Nations_Goblins,4-4
166307,341,Candyland_Liches,6-2
166308,315,Web-of-Nations_Goblins,4-4


In [31]:
#What about Series?
query = """
SELECT * FROM Series LIMIT 10;
"""
QUERY(query, is_select=True)

,series_id,winning_team_id,losing_team_id,winner_name,loser_name,series_score,_0N_score,_1N_score,_2N_score,_3N_score,_4N_score,_5N_score,_6N_score,_7N_score,_8N_score,series_game_score,context
0,1,1,8,(9)Universal_Arch-Kings,(10)Universal_Revenants,30-16,24-22,26-20,17-29,27-19,30-16,25-20,24-13,12-12,10-5,195-156,S0 Universal Play-In
1,2,2,13,(7)Universal_Aether,(8)Universal_Incubi,30-26,28-28,23-33,24-32,25-31,33-23,24-30,24-24,23-8,6-6,210-215,S0 Universal Play-In
2,3,13,1,(8)Universal_Incubi,(9)Universal_Arch-Kings,35-20,38-17,24-31,23-32,36-19,23-32,17-36,35-13,15-23,16-2,227-205,S0 Universal Play-In
3,4,26,13,(1)**Cosmic_Slashers**,(8)Universal_Incubi,75-4,60-19,43-36,63-16,53-26,57-22,52-17,29-15,21-6,11-2,389-159,S0 R1W Universal Playoffs
4,5,6,11,(5)Universal_Seraphs,(4)Universal_Voidwalkers,92-72,98-66,70-94,77-87,73-91,91-73,60-94,81-52,49-37,34-21,633-615,S0 R1W Universal Playoffs
5,6,18,22,(3)Universal_Zealots,(6)Universal_Zombies,75-18,56-37,67-26,50-43,63-30,55-38,55-31,40-28,25-17,15-5,426-255,S0 R1W Universal Playoffs
6,7,3,2,(2)Universal_Oracle,(7)Universal_Aether,75-4,57-22,63-16,50-29,61-18,61-18,52-11,25-6,13-2,3-1,385-123,S0 R1W Universal Playoffs
7,8,11,13,(4)Universal_Voidwalkers,(8)Universal_Incubi,75-43,56-62,68-50,81-37,64-54,70-48,72-42,47-43,26-29,15-18,499-383,S0 R1L Universal Playoffs
8,9,22,2,(6)Universal_Zombies,(7)Universal_Aether,160-140,149-151,179-121,164-136,131-169,134-166,164-126,126-129,94-90,43-39,1184-1127,S0 R1L Universal Playoffs
9,10,26,6,(1)**Cosmic_Slashers**,(5)Universal_Seraphs,85-24,59-50,71-38,70-39,65-44,76-33,50-46,52-28,37-15,15-9,495-302,S0 R2W Universal Playoffs


In [32]:
#I want to see the series where the 1 seed was upset.
query = """
SELECT * FROM Series WHERE loser_name LIKE '%(1)%' LIMIT 10;
"""
QUERY(query, is_select=True)

,series_id,winning_team_id,losing_team_id,winner_name,loser_name,series_score,_0N_score,_1N_score,_2N_score,_3N_score,_4N_score,_5N_score,_6N_score,_7N_score,_8N_score,series_game_score,context
0,14,3,26,(2)Universal_Oracle,(1)**Cosmic_Slashers**,100-62,96-66,88-74,87-75,99-63,89-73,86-70,70-55,51-44,21-27,687-547,S0 WB Finals Universal Playoffs
1,17,3,26,(2)Universal_Oracle,(1)**Cosmic_Slashers**,200-86,150-136,147-139,133-153,180-106,176-110,166-103,135-97,101-69,53-32,1241-945,S0 Grand Finals Universal Playoffs
2,67,84,71,(4)Diamond-Sea_Exorcists,(1)Diamond-Sea_Ghost-Runners,76-60,80-56,76-60,75-61,84-52,72-64,68-59,45-67,27-53,26-16,553-488,S1 R2W Diamond-Sea Regional Playoffs
3,72,82,71,(3)Diamond-Sea_Ghosts,(1)Diamond-Sea_Ghost-Runners,68-50,60-58,56-62,56-62,60-58,56-62,57-58,56-41,35-32,28-11,464-444,S1 LB Finals Diamond-Sea Regional Playoffs
4,166,170,184,(2)Steel-Heart_Seraphs,(1)Steel-Heart_Inquisitors,65-19,67-17,59-25,40-44,44-40,25-59,18-64,54-24,36-21,31-4,374-298,S1 WB Finals Steel-Heart Regional Playoffs
5,169,170,184,(2)Steel-Heart_Seraphs,(1)Steel-Heart_Inquisitors,90-35,95-30,78-47,51-74,76-49,51-74,31-87,78-29,47-31,30-6,537-427,S1 Grand Finals Steel-Heart Regional Playoffs
6,364,254,240,(2)Darkwing_Dryads,(1)Darkwing_Devils,65-35,57-43,68-32,59-41,68-32,44-56,39-51,40-32,19-30,21-8,415-325,S1 WB Finals Darkwing Regional Playoffs
7,367,254,240,(2)Darkwing_Dryads,(1)Darkwing_Devils,104-85,121-68,124-65,90-99,104-85,80-109,85-98,74-77,38-75,40-23,756-699,S1 Grand Finals Darkwing Regional Playoffs
8,693,401,261,(4)House-of-Achlys_Clerics,(1)Shining-Core_Oracle,55-28,58-25,43-40,32-51,54-29,35-48,26-52,50-16,31-20,17-9,346-290,S2 R2W Shining-Core Regional Playoffs
9,698,275,261,(2)Shining-Core_Dwarves,(1)Shining-Core_Oracle,65-34,62-37,56-43,53-46,65-34,52-47,58-33,31-44,23-31,20-9,420-324,S2 LB Finals Shining-Core Regional Playoffs


In [34]:
#Has this ever happened in the Universal Playoffs?
query = """
SELECT winner_name, loser_name, series_score, context FROM Series WHERE loser_name LIKE '%(1)%' AND context LIKE '%Universal%' AND CONTEXT NOT LIKE '%Relegation%';
"""
QUERY(query, is_select=True)

,winner_name,loser_name,series_score,context
0,(2)Universal_Oracle,(1)**Cosmic_Slashers**,100-62,S0 WB Finals Universal Playoffs
1,(2)Universal_Oracle,(1)**Cosmic_Slashers**,200-86,S0 Grand Finals Universal Playoffs
2,(4)Universal_Warlocks,(1)Shining-Core_Wyverns,1701-1590,S6 Universal Finals


These are only basic queries of course, and at the moment I'm working on PlayerSeason and TeamSeason tables so that I can see the statistics of each of them per season.
More advanced queries and updates will be coming.